In [1]:
import os
import numpy as np
from os import listdir
from os.path import isfile, join
import h5py

extiontion = 'txt'

# Path to the parent dir of directories containing corresponding activations for images (one file per image)
# You can move all the image files to a directory and then provide the parent directory path here.
image_dir = "/lclhome/mazim003/Desktop/PythonWorkspace/Nexus/images_original_size/bottleneck_FC"
out_file_name = "Nexus"
google_images_flag = False # True if the images are the images of Google dataset
aloi_images_flag = False # True if the images are the images of Aloi dataset

sub_dirs = [x[0] for x in os.walk(image_dir)]
print(sub_dirs)

# # one time for creating illumination_holdout ALOI dataset where we need the class labels for ALOI images
# labels = []
# for index in range(1000):
#     labels.extend([index] * 24)
# orders = range(1,24001)

for sub_dir in sub_dirs:
    dir_name = os.path.basename(sub_dir)
    if sub_dir == image_dir:
      continue
    print("Looking for images in '" + sub_dir + "'")
    
    # sorting for ALOI & Nexus dataset (sorting determins the image labels for Nexus dataset. 
    # The labels for Google/ALOI are not important as they are all attack samples.)
    image_files = [f for f in listdir(sub_dir) if (isfile(join(sub_dir, f)) and f.endswith(".txt"))  ]
    image_files.sort(key=lambda x: '{0:0>70}'.format(x).lower())
    image_files = np.array(image_files)
    
    # sorting for google
    if google_images_flag:
        image_files.sort()
    
    # shuffle images if aloi/google then we split the images to test/train
    if google_images_flag or aloi_images_flag:
        np.random.seed(0)
        np.random.shuffle(image_files)
#         # one time for creating illumination_holdout ALOI dataset where we need the class labels for ALOI images
#         np.random.seed(0)
#         np.random.shuffle(labels)
#         np.random.seed(0)
#         np.random.shuffle(orders)
    
    with open(image_dir + "/List_files_order" + dir_name + ".txt", "a") as f:
       f.write("\n".join(image_files))
#   # one time for creating illumination_holdout ALOI dataset where we need the class labels for ALOI images
#     with open(image_dir + "/labels" + dir_name + ".txt", "a") as f:
#        f.write("\n".join(map(str, labels)))
#     with open(image_dir + "/orders" + dir_name + ".txt", "a") as f:
#        f.write("\n".join(map(str, orders)))
    
    ourputfile = os.path.join(image_dir,  dir_name + '_images.txt')
    out_file = os.path.join(image_dir,  out_file_name + '.h5')
    f = open(ourputfile, 'ab')
    list_arrays = []
    for cnt, img in enumerate(image_files):
        abs_path = os.path.join(sub_dir,  img)
        data = np.loadtxt(abs_path, delimiter=',')
        list_arrays.append(data.tolist())
        list_arrays = np.array(list_arrays)
#         np.savetxt(f, list_arrays)
        
        # store h5py file: as we may run out of memory, we store the records one by one
        data_row = np.array(list_arrays)
        data_row = np.reshape(data_row, (1,-1))
        if not os.path.exists(out_file):
            h5_file = h5py.File(out_file,'w')
            dataset = h5_file.create_dataset('dataset_1',data_row.shape, 'float32', chunks=(100,data_row.shape[1]), maxshape=(None, None))
            dataset[...] = data_row
            h5_file.close()
        else: # if file exists, append to h5py dataset
            h5_file = h5py.File(out_file, 'r+')
            dataset = h5_file["dataset_1"]
            dataset.resize((dataset.shape[0] + 1,dataset.shape[1]))
            dataset[dataset.shape[0] -1] = data_row
            h5_file.close()
        list_arrays = []
        
        if cnt%1000==0:
            print(cnt)
    f.close()
    print("ourputfile %s" %ourputfile)
    print("Finished")


['/lclhome/mazim003/Desktop/PythonWorkspace/ALOI_DirectLight/Images/ALOI_images_jpgs_brighter_1.4/bottleneck_FC', '/lclhome/mazim003/Desktop/PythonWorkspace/ALOI_DirectLight/Images/ALOI_images_jpgs_brighter_1.4/bottleneck_FC/2']
Looking for images in '/lclhome/mazim003/Desktop/PythonWorkspace/ALOI_DirectLight/Images/ALOI_images_jpgs_brighter_1.4/bottleneck_FC/2'
sorted
['1_l1c1.jpg.txt' '1_l1c2.jpg.txt' '1_l1c3.jpg.txt' ...,
 '1000_l8c1.jpg.txt' '1000_l8c2.jpg.txt' '1000_l8c3.jpg.txt']
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
ourputfile /lclhome/mazim003/Desktop/PythonWorkspace/ALOI_DirectLight/Images/ALOI_images_jpgs_brighter_1.4/bottleneck_FC/2_images.txt
Finished
